In [1]:
#Read raw JSON written to S3 by Kinesis

from pyspark.sql.types import *
from pyspark.sql.functions import struct

file_path = "s3://petebchamp-s3-raw/2020"

jsonSchema = (
  StructType()
    .add("createdate", TimestampType())
    .add("firstname", StringType())
    .add("lastname", StringType())
    .add("gender", StringType())
    .add("birthdate", DateType())
)

df = spark.read.json(path = file_path + "/*/*/*/*", schema = jsonSchema)
df.show(10)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1596321508721_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+--------+------+----------+
|          createdate|firstname|lastname|gender| birthdate|
+--------------------+---------+--------+------+----------+
|2020-07-29 18:19:...|     Cory|   Parks|     M|2013-02-01|
|2020-07-29 18:19:...|    David|  Cooper|     F|2001-09-02|
|2020-07-29 18:19:...|    Kayla|  Chavez|     F|1994-10-29|
|2020-07-29 18:19:...|    James| Johnson|     F|1929-07-13|
|2020-07-29 18:19:...|     Adam| Garrett|     M|1998-01-07|
|2020-07-29 18:19:...|     Mark|  Adkins|     M|2004-07-30|
|2020-07-29 18:19:...| Kristine|  Flores|     M|1909-11-06|
|2020-07-29 18:20:...|  Kristin|  Garcia|     M|1960-05-23|
|2020-07-29 18:20:...|  Tiffany|  Jensen|     F|1955-08-13|
|2020-07-29 18:20:...|  Matthew|   Yates|     M|1928-02-14|
+--------------------+---------+--------+------+----------+
only showing top 10 rows

In [2]:
df.printSchema

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<bound method DataFrame.printSchema of DataFrame[createdate: timestamp, firstname: string, lastname: string, gender: string, birthdate: date]>

In [3]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

30

In [4]:
# Write to S3 as parquet
# This will create duplicates if run multiple times (which is deliberate in this case, as they will be eliminated below)

dest_path = 's3://petebchamp-s3/parquet_files'
df.write.parquet(path = dest_path, mode = 'append')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df_parquet = spark.read.parquet(dest_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_parquet.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

180

In [7]:
# Originally received error: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient
# Created new cluster with additional applications

df_parquet.createOrReplaceTempView("tbl")

sql = """
    SELECT 
        createdate, firstname, lastname, gender, birthdate, 
        ROW_NUMBER() OVER(PARTITION BY firstname, lastname, gender, birthdate ORDER BY createdate) row_num
    FROM tbl
    """

df_parquet_dupes1 = spark.sql(sql)
df_parquet_clean1 = df_parquet_dupes1.where('row_num = 1')
df_parquet_clean1.count()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

30

In [8]:
#Same output as above

from pyspark.sql.window import *
from pyspark.sql.functions import row_number

df_parquet_dupes2 = (df_parquet
    .withColumn('row_num', row_number()
        .over(Window.partitionBy(['firstname', 'lastname', 'gender', 'birthdate'])
        .orderBy('createdate'))))

df_parquet_clean2 = df_parquet_dupes2.where('row_num = 1')
df_parquet_clean2.count()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

30